In [5]:
!pip install mistralai

In [6]:
!pip install faiss-cpu

In [7]:
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage
import numpy as np
import os

In [8]:
f = open('essay.txt', 'r')
t=f.read()
f.close()

In [9]:

t

'The Indian Independence Act 1947 (10 & 11 Geo. 6. c. 30) is an act of the Parliament of the United Kingdom that partitioned British India into the two new independent dominions of India and Pakistan. The Act received Royal Assent on 18 July 1947 and thus modern-day India and Pakistan, comprising west (modern day Pakistan) and east (modern day Bangladesh) regions, came into being on 15 August.[1][a]\n\nThe legislature representatives of the Indian National Congress,[2] the Muslim League,[3] and the Sikh community[4] came to an agreement with Lord Mountbatten, then Viceroy and Governor-General of India, on what has come to be known as the 3 June Plan or Mountbatten Plan. This plan was the last plan for independence..\n\nPrelude\nAttlee\'s announcement\nClement Attlee, the Prime Minister of the United Kingdom, announced on 20 February 1947 that:\n\nThe British Government would grant full self-government to British India by 30 June 1948 at the latest,\nThe future of the Princely States wo

generating chunks  out of text corpora

In [10]:
import re
# splitting by fullstop
def split_text_into_chunks(text, chunk_size):
    # Split the text into sentences
    sentences = re.split(r'(?<=[.!?]) +', text)

    chunks = []
    current_chunk = ""

    for sentence in sentences:
       
        if len(current_chunk) + len(sentence) + 1 > chunk_size:
            chunks.append(current_chunk)
            current_chunk = sentence
        else:
         
            if current_chunk:
                current_chunk += " " + sentence
            else:
                current_chunk = sentence

   
    if current_chunk:
        chunks.append(current_chunk)

    return chunks


chunk_size = 128
chunks = split_text_into_chunks(t, chunk_size)

print(f"Number of chunks: {len(chunks)}")
for i, chunk in enumerate(chunks):
    print(f"Chunk {i+1}: {chunk}")


Number of chunks: 34
Chunk 1: The Indian Independence Act 1947 (10 & 11 Geo. 6. c.
Chunk 2: 30) is an act of the Parliament of the United Kingdom that partitioned British India into the two new independent dominions of India and Pakistan.
Chunk 3: The Act received Royal Assent on 18 July 1947 and thus modern-day India and Pakistan, comprising west (modern day Pakistan) and east (modern day Bangladesh) regions, came into being on 15 August.[1][a]

The legislature representatives of the Indian National Congress,[2] the Muslim League,[3] and the Sikh community[4] came to an agreement with Lord Mountbatten, then Viceroy and Governor-General of India, on what has come to be known as the 3 June Plan or Mountbatten Plan.
Chunk 4: This plan was the last plan for independence..

Prelude
Attlee's announcement
Clement Attlee, the Prime Minister of the United Kingdom, announced on 20 February 1947 that:

The British Government would grant full self-government to British India by 30 June 1948 at th

In [11]:
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage

# Initialize the Mistral AI client
client = MistralClient(api_key='p3T9odnj2XtqQLiMpAfi06r8ZBUrYEnB')


def get_text_embedding(input):
    embeddings_batch_response = client.embeddings(
        model="mistral-embed",
        input=input
    )
    return embeddings_batch_response.data[0].embedding


text_embeddings = np.array([get_text_embedding(chunk) for chunk in chunks])

print(text_embeddings)


[[-0.03982544  0.05175781  0.02766418 ...  0.01696777  0.01418304
  -0.00453186]
 [-0.02206421  0.03466797  0.05136108 ...  0.01029205  0.02354431
   0.00243568]
 [-0.03936768  0.03186035  0.05050659 ... -0.00821686  0.02185059
  -0.0114975 ]
 ...
 [-0.01774597  0.01589966  0.02697754 ... -0.03842163  0.03762817
   0.00879669]
 [-0.04232788  0.02914429  0.03887939 ... -0.03128052  0.01625061
  -0.00693512]
 [-0.04373169  0.02450562  0.03518677 ...  0.00796509 -0.00762177
  -0.00822449]]


Loading into a vector database

-instantiating an instance of the Index class
-then put the embeddings in index structure

In [21]:
import faiss
d=text_embeddings.shape[1]
index=faiss.IndexFlatL2(d)
index.add(text_embeddings)

creating embeddings out of query


In [33]:
question = "Who started as first governal general of india?"
question_embeddings = np.array([get_text_embedding(question)])

Finds Euclidean distance


In [34]:
D, I = index.search(question_embeddings, k=2)
retrieved_chunk = [chunks[i] for i in I.tolist()[0]]

In [35]:
retrieved_chunk

['Though it was technically possible for the states to remain free after 15 August, their dependence on the British government of India for defence, foreign affairs, communication and other matters rendered such freedom meaningless.[12][13]\n\nIndia\nFurther information: Princely states of India and Political integration of India\nLord Mountbatten continued as the first Governor General of independent India.',
 'Jawaharlal Nehru became the prime minister and Vallabhbhai Patel became the home minister.\n\nOver 550 princely states, almost all of the states contiguous with the territory of India, acceded to India by 15 August.']

In [36]:
prompt = f"""
Context information is below.
---------------------
{retrieved_chunk}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {question}
Answer:
"""

Give this prompt to mistral ai

In [37]:
def run_mistral(user_message, model="mistral-medium-latest"):
    messages = [
        ChatMessage(role="user", content=user_message)
    ]
    chat_response = client.chat(
        model=model,
        messages=messages
    )
    return (chat_response.choices[0].message.content)

run_mistral(prompt)

'Lord Mountbatten started as the first Governor General of independent India.'

In [39]:
def ans(question):
    question_embeddings = np.array([get_text_embedding(question)])

    D, I = index.search(question_embeddings, k=2)

    retrieved_chunk = [chunks[i] for i in I.tolist()[0]]

    prompt = f"""
    Context information is below.
    ---------------------
    {retrieved_chunk}
    ---------------------
    Given the context information and not prior knowledge, answer the query.
    Query: {question}
    Answer:
    """


    def run_mistral(user_message, model="mistral-medium-latest"):
        messages = [
            ChatMessage(role="user", content=user_message)
        ]
        chat_response = client.chat(
            model=model,
            messages=messages
        )
        return chat_response.choices[0].message.content


    return run_mistral(prompt)





In [40]:
question = "Who wants status?"
answer = ans(question)
print(answer)

The Nizam of Hyderabad wants Dominion status.


In [41]:
question = "Who did the invasion on hyderbad and what was it called?"
answer = ans(question)
print(answer)

The Indian government conducted the invasion on Hyderabad, and it was called Operation Polo.
